In [207]:
# predict

from neurotron import Cluster, Matrix, Token, Record, Monitor, Toy, isa, follow
import neurotron.math as nm


In [208]:
class Cells(Cluster):
    """
    >>> Cells()
    |-|-|-|-|-|-|-|-|-|-|
    >>> Cells('Mary')
    """
    def __init__(self,shape=(2,5,2,3),token=None):
        if isa(shape,str):
            tag = shape      # rename
            self.toy = Toy(tag)
            shape = self.toy.shape
            token = self.toy.token
        else:
            self.toy = None
        nm.seed(1)
        cells = super().__init__(*shape,verbose=1)
        self.token = token
        m,n,d,s = shape
        f = [0,0,0] if token is None else token['.']
        self.y = nm.row(nm.zeros(1,m*n),f)
        self.record = Record(self)

    def process(self,seq):
        m,n,d,s = self.shape
        seq = [seq] if isa(seq,str) else seq
        prediction = [seq[0],'->']
        for word in seq:
            mon = Monitor(m,n);
            self.y = nm.row(nm.zeros(1,m*n),token[word])
            self.y = self.step(mon,self.y,word)
            output,predict = self.decode()
            mon.xlabel((n-1)/2,output+' -> '+predict)
            prediction.append(predict)
        return prediction

    def predictive(self,list):
        for k in list:
            cells.X[k] = 1;
            cells._predict.I[k] = Matrix([[.1,-.1,.1,-.1,.1],[0,0,0,0,0]])

    def map(self):
        self._predict.map()
        
    def __str__(self):
        self.record.clear();  
        self.record(self)
        return self.record.pattern()

    def __repr__(self):
        return self.__str__()



In [209]:
#=========================================================================
# setup network
#=========================================================================

token = Token({'Mary': [1,0,0,0,0,0,0,1,1],
               'John': [0,1,0,0,0,0,0,1,1],
               'Lisa': [1,0,0,0,0,0,1,1,0],
               'Andy': [0,1,0,0,0,0,1,1,0],
               'likes':[0,0,1,0,0,0,0,1,1],
               'to':   [0,0,0,1,0,0,0,1,1],
               'sing': [0,0,0,0,1,0,0,1,1],
               'dance':[0,0,0,0,0,1,0,1,1],
               'hike': [0,0,0,0,1,0,1,1,0],
               'climb':[0,0,0,0,1,0,1,0,1],
               '.':    [0,0,0,0,0,0,1,1,1],
              })
    
def setup(token):
    shape = (2,9,8,3);  
    cells = Cells(shape,token)
    cells._predict.clear()

    try:
        cells.connect([0,14,16],[4,14,16])  # Mary -> likes
        cells.connect([4,14,16],[6,14,16])  # likes -> to
        cells.connect([6,14,16],[8,14,16])  # to -> sing
        cells.connect([8,14,16],[12,14,16]) # sing -> .

        cells.connect([2,14,16],[5,15,17])  # John -> likes  
        cells.connect([5,15,17],[7,15,17])  # likes -> to
        cells.connect([7,15,17],[10,15,17]) # to(7,15,17) -> dance(10,15,17)
        cells.connect([10,15,17],[13,15,17])# dance -> .

        cells.connect([0,12,14],[4,14,17])  # Lisa -> likes
        cells.connect([4,14,17],[6,15,16])  # likes -> to
        cells.connect([6,15,16],[8,12,14])  # to -> hike
        cells.connect([8,12,14],[13,15,17]) # hike -> .

        cells.connect([2,12,14],[5,14,17])  # Andy -> likes
        cells.connect([5,14,17],[7,15,16])  # likes -> to
        cells.connect([7,15,16],[9,12,16])  # to -> climb
        cells.connect([9,12,16],[13,15,17]) # climb -> .
    except Synapse as s:
        print(Ansi.R,'*** warning:',str(s),Ansi.N)
        
    return cells

cells = setup(token)


In [230]:
class Train:
    """
    parameter training
    >>> Train()
    Train(Cells(2,5,2,3))
    >>> Train(Cells('Mary')
    """
    def __init__(self,cells=None):
        self.memory = {}
        self._words = {}
        self._contexts = {}
        self.cells = Cells() if cells is None else cells

    def pattern(self,list):
        """
        >>> Train().pattern([1,0,1,0])
        '1010'
        """
        str = ''
        for item in list: str += '1' if item else '0'
        return str
    
    def hash(self,M):    # hash of a matrix
        """
        Train().hash(Matrix([[1,1,0],[0,1,1]]))
        """
        list = M.list();  str = ''
        m,n = M.shape; sep = ''
        for i in range(m):
            row = list[i]
            str += sep + self.pattern(row); sep = '|'
        return str

    def next(self,M):
        """
        >>> Train().next(Matrix(2,3))
        [1 1 1; 0 0 0]
        >>> Train().next([[0,0,0],[1,1,1]])
        [0 0 1; 1 1 0]
        >>> Train().next([[0,1,1],[1,0,0]])
        [1 0 1; 0 1 0]
        """
        if M is None:
            m,n,d,s = self.cells.shape
            return follow(Matrix(m,n))
        return follow(M)

    def token(self,word=None):
        """
        >>> dict = Train(Cells('Mary')).token() 
        >>> Train(Cells('Mary')).token('likes')
        [0, 0, 1, 0, 0, 0, 0, 1, 1]
        """
        if word is None: return self.cells.token
        return self.cells.token[word]

    def number(self,M):
        m,n = M.shape; nmb = 0; base = 1;
        for j in range(n):
            for i in range(m):
                if M[i,j]:
                    nmb += base*i; break
            base *= m
        return nmb
        
    def index(self,token):
        """
        >>> train = Train(Cells('Mary'))
        >>> train.index(train.token('likes'))
        [2, 7, 8]
        """
        idx = []
        for k in range(len(token)):
            if token[k]: idx.append(k)
        return idx

    def _word(self,word):  # store/update word to ._words
        """
        word 'Mary' is stored as ([0,7,8],'#0',[[1,1,1],[0,0,0]])
        >>> Train(Cells('Mary'))._update_word('Mary')
        ('Mary',([0,7,8],'#0',[[1,1,1],[0,0,0]]))
        """
        m = self.cells.shape[0]
        n = len(self.index(self.token(word)))
        if not word in self._words:
            triple = (self.index(self.token(word)),'#-1',Matrix(m,n))
            #print('### triple:',triple)
        else:
            triple = self._words[word]
            idx,key,M = triple 
            key = '#%g' % (int(key[1:])+1)
            M = self.next(M)
            if M is None: 
                raise Exception('representation overflow (max %g)' % m**n)
            triple = (idx,key,M)
        self._words[word] = triple
        return(word,triple)
        
    def _learn(self,context,word):    # process first item of sequence
        if context == '':
            context = word
        else:
            context = context + ' ' + word
        value = self.memory[context] if context in self.memory else []
        print('### learn: ',context,'->',value)
        idx = self.index(self.token(word))
        if value == []:
            value.append((word,idx))
            self.memory[context] = value
        print('           ',context,'->',self.memory[context])
        return context
        
    def _train(self,context,word):
        """
        >>> train = Train(Cells('Mary'))
        """
        print('### train:',(context,word))
        if context == '':
            if not word in self._words:
                self._word(word)
            return self._words[word]
        context = self._learn(context,word)
        return context

    def __call__(self,context,word):
        return self._train(context,word)
        
    def __str__(self):
        if self.cells is None: return 'Train()'
        if self.cells.toy is not None:
            return "Train(Cells('%s'))" % self.cells.toy.tag
        return 'Train(Cells(%g,%g,%g,%g))' % self.cells.shape

    def __repr__(self):
        return self.__str__()
"""
how to represent 'Mary'?
=> from token get the columns (indices)
'Mary':  ([0,7,8],#0,[[1,1,1],[0,0,0]])
'John':  ([1,7,8],#0,[[1,1,1],[0,0,0]])
'likes': ([2,7,8],#2,[[1,0,1],[0,1,0]])
'to':    ([3,7,8],#0,[[1,1,1],[0,0,0]])
'sing':  ([4,7,8],#0,[[1,1,1],[0,0,0]])
'paint': ([5,6,7],#0,[[1,1,1],[0,0,0]])
'<Mary>': {
    '#':([0 7 8],'#0','Mary')
    '@': ['#0',[1 1 1; 0 0 0],'0.0-7.0-8.0']
    'likes':'<Mary likes>'
    }
'<John>': {
    '#':([1,7,8],'#0','John')
    '@': ['#0',[1 1 1; 0 0 0],'1.0-7.0-8.0']
    'likes':'<John likes>'
    }
'<Mary likes>': {
    '#':([2,7,8],'#0','likes')
    '@':['#0',[1 1 1; 0 0 0],'2.0-7.0-8.0'],
    'to': '<Mary likes to>'
    }
'<John likes>': {
    '#':([2,7,8],'#1','likes')
    '@':['#1',[0 1 1; 1 0 0],'2.1-7.0-8.0'],
    'to': '<John likes to>'
    }
'Lisa likes': {
    '#':([2,7,8],'#2','likes')
    '@':['#2',[1 0 1; 0 1 0],'2.0-7.1-8.0'],
    'to': '<Lisa likes to>'
    }
'Mary likes to': { 
    '#':([3,7,8],'#0','to')
    '@':['#0',[1 1 1; 0 0 0],'2.0-7.0-8.0'],
    'sing': [3,'Mary likes to sing']
    'paint': [1,'Mary likes to paint']
    }
'John likes to': { 
    '#':([3,7,8],'#1','to')
    '@':['#1',[0 1 1; 1 0 0],'2.1-7.0-8.0'],
    'sing': [2,'<John likes to sing>']
    'climb': [1,'<John likes to climb>']
    }

'Mary likes to sing': { 
    '#':([4,7,8],'#0','sing')
    '@':['#0',[1 1 1; 0 0 0],'4.0-7.0-8.0'],
    '.': [3,'<Mary likes to sing .>']
    }

'Mary likes to paint': { 
    '#':([5,6,7],'#0','paint'],
    '@':['#0',[1 1 1; 0 0 0],'5.0-6.0-7.0'],
    '.': [1,'Mary likes to paint .']
    }
    
"""
[]       

[]

In [231]:
train = Train(Cells('Mary')); print(train)
train.next(Matrix(2,3))

Train(Cells('Mary'))


[1 1 1; 0 0 0]

In [232]:
context = train('','Mary'); print('context:',context)
context = train('','Mary'); print('context:',context)
context = train('','Mary'); print('context:',context)

### train: ('', 'Mary')
context: ([0, 7, 8], '#-1', [0 0 0; 0 0 0])
### train: ('', 'Mary')
context: ([0, 7, 8], '#-1', [0 0 0; 0 0 0])
### train: ('', 'Mary')
context: ([0, 7, 8], '#-1', [0 0 0; 0 0 0])


In [233]:
context = train('Mary','likes'); print('context:',context)

### train: ('Mary', 'likes')
### learn:  Mary likes -> []
            Mary likes -> [('likes', [2, 7, 8])]
context: Mary likes


In [234]:
train.index(train.token('Mary'))

[0, 7, 8]

In [235]:
list=[1,2]

In [236]:
list[1]=8;list

[1, 8]

In [242]:
for (k in range(3): print(train('','Mary')); train._words

### train: ('', 'Mary')
([0, 7, 8], '#7', [0 0 0; 1 1 1])


{'Mary': ([0, 7, 8], '#7', [0 0 0; 1 1 1])}

In [240]:
train = Train(Cells('Mary'))
for k in range(66):
    print(train._word('Mary')); train._words

('Mary', ([0, 7, 8], '#-1', [0 0 0; 0 0 0]))
('Mary', ([0, 7, 8], '#0', [1 1 1; 0 0 0]))
('Mary', ([0, 7, 8], '#1', [0 1 1; 1 0 0]))
('Mary', ([0, 7, 8], '#2', [1 0 1; 0 1 0]))
('Mary', ([0, 7, 8], '#3', [0 0 1; 1 1 0]))
('Mary', ([0, 7, 8], '#4', [1 1 0; 0 0 1]))
('Mary', ([0, 7, 8], '#5', [0 1 0; 1 0 1]))
('Mary', ([0, 7, 8], '#6', [1 0 0; 0 1 1]))
('Mary', ([0, 7, 8], '#7', [0 0 0; 1 1 1]))


Exception: representation overflow (max 8)

In [239]:
train.next(Matrix(2,3))

[1 1 1; 0 0 0]